In [94]:
# this are all the dependencies we need
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
import os
import json
from api_keys import g_key
import pandas as pd

In [95]:
# let us acess gmaps with the key
gmaps.configure(api_key=g_key)

In [96]:
# we are going to bring the data from the step 1 into a dataframe
cities_dataframe = pd.read_csv("clean_cities_data.csv")
cities_dataframe.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Kapoeta,4.7717,33.5903,72.75,59,67,4.85,SS,1627962088
1,Nueva Granada,9.8017,-74.3930,74.97,93,99,6.22,CO,1627962088
2,Maroantsetra,-15.4333,49.7333,65.10,87,46,3.53,MG,1627962088
3,Omboué,-1.5746,9.2618,73.76,89,96,4.21,GA,1627961857
4,Gornopravdinsk,60.0500,69.9000,57.49,96,100,7.05,RU,1627962089


In [97]:
# we are going to get the latitude and longitude separated
coordinates = cities_dataframe[["Lat","Lng"]]

In [98]:
# get humidity
humidity = cities_dataframe["Humidity"]

In [99]:
# get temperature
maximum_temperature = cities_dataframe[["Max Temp"]]

In [100]:
# we need to create the layout and figure
layout = {
    'width': '2000px',
    'height': '1000px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
figure = gmaps.figure()


In [101]:
# create the heatmap layout
heatmap_layout = gmaps.heatmap_layer(coordinates, weights = humidity,
                  dissipating=False, max_intensity=300,
                  point_radius=3, opacity = 0.60)
markings = gmaps.marker_layer(coordinates)

In [102]:
# display the figure
figure.add_layer(heatmap_layout)
figure

Figure(layout=FigureLayout(height='420px'))

In [109]:
# let us reduce the dataframe to fit the following
# A max temperature lower than 80 degrees, but higher than 70
cities_dataframe_80_70 = (cities_dataframe["Max Temp"] < 80) & (cities_dataframe["Max Temp"] > 70)

In [137]:
# Wind speed less than 10 mph
cities_wind = cities_dataframe["Wind Speed"] < 10

In [138]:
# Zero cloudiness
cities_cloud = cities_dataframe.Cloudiness == 0
# see final dataframe
final_dataframe = cities_dataframe_80_70 & cities_wind & cities_cloud

In [139]:
# final touches
adjusted_dataframe = cities_dataframe[final_dataframe]
adjusted_dataframe = adjusted_dataframe.dropna()
adjusted_dataframe = adjusted_dataframe.reset_index()
adjusted_dataframe.head()

,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,105,Mitsamiouli,-11.3847,43.2844,75.58,69,0,4.79,KM,1627962117
1,131,Batagay-Alyta,67.8006,130.4114,73.99,22,0,8.41,RU,1627962125
2,155,Poum,-20.2333,164.0167,70.36,66,0,9.24,NC,1627962131
3,200,Boa Vista,2.8197,-60.6733,76.98,88,0,2.30,BR,1627962144
4,226,Abu Samrah,35.3029,37.1841,79.61,44,0,6.38,SY,1627962152


In [140]:
adjusted_dataframe.columns

Index(['index', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')

In [141]:
# we want to find the first hotel near each city within 5000 meters
hotel_dataframe_stored = adjusted_dataframe
hotel_dataframe_stored["Hotel Name"] = ""

In [142]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
parameters = {
        "keyword": "hotel",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }

# loop that will find the location based on the coordinates given
for index, row in hotel_dataframe_stored.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    parameters["location"]=f"{lat},{lng}"

# creating a request
    hoteldata = requests.get(base_url, parameters).json()

# this will be our exception  
    try:
        hotelname= hoteldata["results"][0]["name"]
        hotel_dataframe_stored.loc[index, "Hotel Name"]= hotelname
        print(hotelname)     
    except:
        hotel_dataframe_stored.loc[index,"Hotel Name"]= np.nan
hotel_dataframe_stored.dropna(inplace=True)
# print results        
hotel_dataframe_stored

Al Camar Lodge
Sberbank
Hotel Orla do Rio Branco
Villa Estet by Anemon Yanıkülke bağ otel
Blue Mudanya Hotel


,index,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,105,Mitsamiouli,-11.3847,43.2844,75.58,69,0,4.79,KM,1627962117,Al Camar Lodge
1,131,Batagay-Alyta,67.8006,130.4114,73.99,22,0,8.41,RU,1627962125,Sberbank
3,200,Boa Vista,2.8197,-60.6733,76.98,88,0,2.30,BR,1627962144,Hotel Orla do Rio Branco
5,297,Kula,38.5473,28.6498,74.12,31,0,4.00,TR,1627962173,Villa Estet by Anemon Yanıkülke bağ otel
9,505,Mudanya,40.3753,28.8822,77.09,37,0,4.97,TR,1627962233,Blue Mudanya Hotel


In [151]:
# Now we want to plot the hotels on the humidity heatmap with hotel names, city, and country

# here we add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_dataframe_stored.iterrows()]

#Create a map using coordinates to set markers
marker_locations = hotel_dataframe_stored[["Lat", "Lng"]]

In [152]:
# Add marker layer for Hotel map
figure = gmaps.figure()

markings = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)

figure.add_layer(heatmap_layout)
figure.add_layer(markings)

# Display figure
figure

Figure(layout=FigureLayout(height='420px'))